In [1]:
%reload_ext autoreload
%autoreload 2

import os

os.chdir(f"/home/{os.getlogin()}/watttime-python-client-aer-algo")

import math
import numpy as np
import pandas as pd
import datetime
from pytz import UTC, timezone
from datetime import datetime, timedelta
import concurrent.futures
import contextlib
import io

from watttime import WattTimeForecast, WattTimeHistorical, RecalculatingWattTimeOptimizer

import data.s3 as s3u
import evaluation.eval_framework as efu

username = os.getenv("WATTTIME_USER")
password = os.getenv("WATTTIME_PASSWORD")

actual_data = WattTimeHistorical(username, password)
hist_data = WattTimeForecast(username, password)

s3 = s3u.s3_utils()
pd.options.mode.chained_assignment = None  # default='warn'
import random
from tqdm import tqdm
import warnings
from plotnine import *
pd.set_option('display.max_columns', None)

In [2]:
dates_2023 = efu.generate_random_dates(2023)

kwargs = {
    "max_percent_capacity": 0.95,  # highest level of charge achieved by battery
    "user_charge_tolerance": 0.8,  # user preferred min charge
    "power_output_efficiency": 1,  # power loss. 1 = no power loss.
    "average_battery_starting_capacity": 0.5,  # average starting percent charged
    "start_hour": "01:00:00",  # earliest session can start
    "end_hour": "23:00:00",  # latest session can start
    "proportion_contiguous" : .2
}

synthetic_data = efu.execute_synth_data_process(
    distinct_date_list=dates_2023, number_of_users=50, **kwargs
)

synthetic_data["session_end_time"] = pd.to_datetime(synthetic_data["session_end_time"], utc=True)
synthetic_data["session_start_time"] = pd.to_datetime(synthetic_data["session_start_time"], utc=True)
synthetic_data = synthetic_data.sample(20)

100%|██████████| 50/50 [00:00<00:00, 53.71it/s]


In [3]:
synthetic_data

,index,distinct_dates,user_type,session_start_time,session_end_time,initial_charge,total_seconds_to_95,full_charge_time,length_of_session_in_seconds,charged_kWh_actual,final_perc_charged,final_charge_time,uncharged,total_capacity,power_output_rate,total_intervals_plugged_in,charge_MWh_needed,charged_MWh_actual,MWh_fraction,usage_time_required_minutes,contiguous_block,charge_per_interval
4665,89,2023-11-10,r19.558_tc42_avglc23770_sdlc7376_contFalse,2023-11-10 01:22:41+00:00,2023-11-10 06:22:17.275451+00:00,0.519102,3331.207159,2023-11-10 02:18:12.207158642,17976.275451,18.097708,0.950000,2023-11-10 02:18:12.207158642,False,42,19.5580,59.920918,0.018098,0.018098,0.001630,56.0,False,None
5165,69,2023-09-03,r12.451999999999998_tc92_avglc25434_sdlc7907_c...,2023-09-03 19:09:31+00:00,2023-09-04 01:27:12.024687+00:00,0.566222,10207.768586,2023-09-03 21:59:38.768586493,22661.024687,35.307537,0.950000,2023-09-03 21:59:38.768586493,False,92,12.4520,75.536749,0.035308,0.035308,0.001038,171.0,False,None
3574,38,2023-05-18,r7.986_tc57_avglc20917_sdlc7175_contFalse,2023-05-18 01:21:47+00:00,2023-05-18 09:00:50.218681+00:00,0.557214,10092.634296,2023-05-18 04:09:59.634295925,27543.218681,22.388827,0.950000,2023-05-18 04:09:59.634295925,False,57,7.9860,91.810729,0.022389,0.022389,0.000665,169.0,False,None
124,20,2023-03-14,r3.8554000000000004_tc26_avglc27331_sdlc6915_c...,2023-03-14 10:46:07+00:00,2023-03-14 14:45:59.617910+00:00,0.517371,10503.205137,2023-03-14 13:41:10.205137331,14392.617910,11.248349,0.950000,2023-03-14 13:41:10.205137331,False,26,3.8554,47.975393,0.011248,0.011248,0.000321,176.0,False,None
2680,80,2023-10-10,r5.920000000000001_tc105_avglc28322_sdlc6876_c...,2023-10-10 21:25:49+00:00,2023-10-11 02:54:37.213311+00:00,0.538768,26257.728770,2023-10-11 04:43:26.728769856,19728.213311,32.441951,0.847739,2023-10-11 02:54:37.213311000,False,105,5.9200,65.760711,0.043179,0.032442,0.000493,329.0,False,None
3987,35,2023-05-07,r7.018_tc24_avglc21423_sdlc7320_contFalse,2023-05-07 08:33:29+00:00,2023-05-07 11:54:08.429030+00:00,0.520933,5282.328071,2023-05-07 10:01:31.328070660,12039.429030,10.297605,0.950000,2023-05-07 10:01:31.328070660,False,24,7.0180,40.131430,0.010298,0.010298,0.000585,89.0,False,None
3921,73,2023-09-16,r7.766_tc96_avglc29371_sdlc7630_contFalse,2023-09-16 01:39:33+00:00,2023-09-16 10:52:11.878841+00:00,0.576015,16642.965264,2023-09-16 06:16:55.965264083,33158.878841,35.902575,0.950000,2023-09-16 06:16:55.965264083,False,96,7.7660,110.529596,0.035903,0.035903,0.000647,278.0,False,None
2109,29,2023-04-11,r17.138_tc100_avglc22559_sdlc7873_contTrue,2023-04-11 08:59:01+00:00,2023-04-11 17:20:28.647930+00:00,0.591818,7523.954911,2023-04-11 11:04:24.954911379,30087.647930,35.818205,0.950000,2023-04-11 11:04:24.954911379,False,100,17.1380,100.292160,0.035818,0.035818,0.001428,126.0,True,[126.0]
32,32,2023-04-26,r18.304_tc34_avglc23581_sdlc7086_contFalse,2023-04-26 15:00:33+00:00,2023-04-26 20:55:05.759588+00:00,0.513718,2917.443053,2023-04-26 15:49:10.443053003,21272.759588,14.833577,0.950000,2023-04-26 15:49:10.443053003,False,34,18.3040,70.909199,0.014834,0.014834,0.001525,49.0,False,None
1779,11,2023-02-10,r8.723_tc23_avglc25179_sdlc6943_contFalse,2023-02-10 14:45:54+00:00,2023-02-10 22:14:42.410389+00:00,0.520440,4077.449481,2023-02-10 15:53:51.449480954,26928.410389,9.879887,0.950000,2023-02-10 15:53:51.449480954,False,23,8.7230,89.761368,0.009880,0.009880,0.000727,68.0,False,None


In [4]:
increments = [15, 30, 60, 120]

regions = [
 'CAISO_NORTH',
 'SPP_TX',
 'ERCOT_EASTTX',
 'FPL',
 'SOCO',
 'PJM_CHICAGO',
 'LDWP',
 'PJM_DC',
 'NYISO_NYC'
]

In [5]:
efu.main_requery_results(regions, synthetic_data, increments, "requery/test.parquet", "requery/synthetic_data_test.parquet")

  0%|          | 0/9 [00:00<?, ?it/s]

Processing CAISO_NORTH...
Processing PJM_CHICAGO...Processing SOCO...Processing SPP_TX...Processing ERCOT_EASTTX...Processing LDWP...
Processing FPL...
Processing PJM_DC...




Error processing region FPL, session start 2023-10-27 02:44:14+00:00: Solution not found!
Error processing region SPP_TX, session start 2023-10-27 02:44:14+00:00: Solution not found!
Error processing region PJM_CHICAGO, session start 2023-10-27 02:44:14+00:00: Solution not found!
Error processing region PJM_DC, session start 2023-10-27 02:44:14+00:00: Solution not found!


 11%|█         | 1/9 [01:43<13:49, 103.63s/it]

Processing NYISO_NYC...
Error processing region LDWP, session start 2023-10-27 02:44:14+00:00: Solution not found!


100%|██████████| 9/9 [02:39<00:00, 17.69s/it] 


Successful S3 put_object response. Status - 200
Successful S3 put_object response. Status - 200


In [6]:
s3.load_parquetdataframe("requery/test.parquet")

,point_time,actual_moer,usage,emissions_co2e_lb,energy_usage_mwh,increment,pred_moer,actual_emissions,init_time,user_type,distinct_dates,region
0,2023-11-10 01:20:00+00:00,1414.0,5.0,2.304584,0.00163,Ideal,1414.0,2.304584,2023-11-10 01:22:41+00:00,r19.558_tc42_avglc23770_sdlc7376_contFalse,2023-11-10,SPP_TX
1,2023-11-10 01:25:00+00:00,1419.0,5.0,2.312733,0.00163,Ideal,1419.0,2.312733,2023-11-10 01:22:41+00:00,r19.558_tc42_avglc23770_sdlc7376_contFalse,2023-11-10,SPP_TX
2,2023-11-10 01:30:00+00:00,1417.0,5.0,2.309474,0.00163,Ideal,1417.0,2.309474,2023-11-10 01:22:41+00:00,r19.558_tc42_avglc23770_sdlc7376_contFalse,2023-11-10,SPP_TX
3,2023-11-10 01:35:00+00:00,1433.0,0.0,0.000000,0.00000,Ideal,1433.0,0.000000,2023-11-10 01:22:41+00:00,r19.558_tc42_avglc23770_sdlc7376_contFalse,2023-11-10,SPP_TX
4,2023-11-10 01:40:00+00:00,1432.0,0.0,0.000000,0.00000,Ideal,1432.0,0.000000,2023-11-10 01:22:41+00:00,r19.558_tc42_avglc23770_sdlc7376_contFalse,2023-11-10,SPP_TX
...,...,...,...,...,...,...,...,...,...,...,...,...
101803,2023-03-23 10:40:00+00:00,884.0,0.0,0.000000,0.00000,Requery 120 minutes,894.8,0.000000,2023-03-22 22:19:12+00:00,r5.8608_tc49_avglc21247_sdlc7893_contFalse,2023-03-22,NYISO_NYC
101804,2023-03-23 10:45:00+00:00,882.0,0.0,0.000000,0.00000,Requery 120 minutes,894.8,0.000000,2023-03-22 22:19:12+00:00,r5.8608_tc49_avglc21247_sdlc7893_contFalse,2023-03-22,NYISO_NYC
101805,2023-03-23 10:50:00+00:00,896.0,0.0,0.000000,0.00000,Requery 120 minutes,897.1,0.000000,2023-03-22 22:19:12+00:00,r5.8608_tc49_avglc21247_sdlc7893_contFalse,2023-03-22,NYISO_NYC
101806,2023-03-23 10:55:00+00:00,895.0,0.0,0.000000,0.00000,Requery 120 minutes,897.5,0.000000,2023-03-22 22:19:12+00:00,r5.8608_tc49_avglc21247_sdlc7893_contFalse,2023-03-22,NYISO_NYC


In [7]:
bad_dat = synthetic_data.query("session_start_time == '2023-10-27 02:44:14+00:00'")
region = "FPL"

full_forecast = s3.load_parquetdataframe(f"complete_2023_forecast_history/{region}.parquet").drop_duplicates()
full_forecast['point_time'] = pd.to_datetime(full_forecast['point_time'], utc=True)
full_history = s3.load_parquetdataframe(f"complete_2023_actual_history/{region}.parquet").drop_duplicates()

/tmp/ipykernel_1756970/4084374959.py:1: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns, UTC] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.


In [8]:
bad_dat['power_output_rate'].iloc[0]

np.float64(13.948)

In [11]:
schedules = efu.full_requery_sim(
    region=region,
    full_forecast=full_forecast,
    full_history=full_history,
    increments=increments,
    start_time=bad_dat['session_start_time'].iloc[0],
    end_time=bad_dat['session_end_time'].iloc[0],
    usage_power_kw=13.948,
    time_needed=bad_dat['usage_time_required_minutes'].iloc[0],
    method="auto",
    charge_per_interval=bad_dat["charge_per_interval"].iloc[0]
)

Using contiguous optimizer
== Fixed contiguous fit! ==
Cumulative charge [0, 36]
Using contiguous optimizer
== Baseline fit! ==
Using contiguous optimizer
== Fixed contiguous fit! ==
Cumulative charge [0, 36]
18
Using contiguous optimizer
== Fixed contiguous fit! ==
Cumulative charge [0, 36]
== Fixed contiguous fit! ==
Cumulative charge [0, 36]
[[ 0. nan]
 [ 0. nan]
 [ 0. nan]
 [ 0. nan]
 [ 0. nan]
 [ 0. nan]
 [ 0. nan]
 [ 0. nan]
 [ 0. nan]
 [ 0. nan]
 [ 0. nan]
 [ 0. nan]
 [ 0. nan]
 [ 0. nan]
 [ 0. nan]
 [ 0. nan]
 [ 0. nan]
 [ 0. nan]
 [ 0. nan]]


Exception: Solution not found!

In [12]:
schedules

NameError: name 'schedules' is not defined